In [4]:
import pandas
import matplotlib.pyplot as plt
import numpy as np

def getStationary(df):
  df['delta'] = (df['open'] / df ['close']) - 1
  df['amplitude'] = (df['high'] / df['low']) - 1
  df['close_over_low'] = (df['close'] / df ['low']) - 1
  del df['open']
  del df['high']
  del df['low']
  df['volume'] = df['volume']
  df['close'] = df['close']
  return df

In [5]:
df_btc_22 = ((pandas.read_csv('../data/btc22.csv',sep=',')).sort_values('date')[['date','open','high','low','close','volume']]).set_index('date')
df_btc_22 = getStationary(df_btc_22)

In [6]:
df_btc_22.to_csv('../data/btc22clean.csv')